In [2]:
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable,lpDot
import numpy as np


def soma_ponderada(n,A,b,C,d,coef_func,vetor_lambda):
    model = LpProblem(name="large-problem", sense=LpMaximize)
   # Neste trabalho será tratado apenas dos casos lineares com variáveis reais

    x = [LpVariable(name=f"x{i}", lowBound=0) for i in range(n)]
    # Restrições <=
    if isinstance(A, (list, np.ndarray)) and isinstance(b, (list, np.ndarray)):
        rest1 = A.shape[0]
    
    # Criamos Uma lista que contêm todas as variáveis do problema.

        for i in range(rest1):
            model += (lpDot(A[i], x) <= b[i], f"restrição {i}")
    else:
        pass
    if isinstance(A, (list, np.ndarray)) and isinstance(b, (list, np.ndarray)):
        rest2 = C.shape[0]
        
        # Restrições >=
        
        for i in range(rest2):
            model += (lpDot(C[i], x) >= d[i], f"restrição {i + rest1}")
    else:
        pass

    # Funções Objetivas
    

    funcao_principal =  np.transpose(vetor_lambda)@coef_func@x
    model += funcao_principal
    f_values = coef_func@x


    

    status = model.solve()
    sol = np.zeros(n)
    for i in range(n):
        sol[i] = x[i].value()
    print(f"Solução: {sol}")
    sol = np.transpose(sol)
    value_func_aux = np.zeros(coef_func.shape[0])
    for i in range(coef_func.shape[0]):
     value_func_aux[i] = np.dot(sol,coef_func[i, :])
    for i in range(coef_func.shape[0]):
        print(f"O valor de f{i+2} é: {value_func_aux[i]}")



    print(f"status: {model.status}, {LpStatus[model.status]}")
    #print(f"Objetivo: {model.objective.value()}")
   # print(model)

In [20]:
n=2
A = np.array([[1,3]])
b = np.array([3])
C = np.array([[1,1]])
d=np.array([1])
coef_func=np.array([[1,1],[2,1]])
vetor_lambda = np.array([1,2])

soma_ponderada(n,A,b,C,d,coef_func,vetor_lambda)

Solução: [3. 0.]
O valor de f2 é: 3.0
O valor de f3 é: 6.0
status: 1, Optimal
Objetivo: 15.0
large-problem:
MAXIMIZE
5*x0 + 3*x1 + 0
SUBJECT TO
restrição_0: x0 + 3 x1 <= 3

restrição_1: x0 + x1 >= 1

VARIABLES
x0 Continuous
x1 Continuous



In [73]:
"""
n=6
A = np.array([[216,164,456,516,176,670],[28.1,13.6,0,0,5,0],[1.6,8.5,0,0,0,0],[166.1,167.1,205.1,209.1,236.1,228.1],[2.05,2.1,8.35,11.95,8.2,13.8]])
d=np.array([1800,100,25,1300,22])
C = A
#d=np.array([100,22,1500,25,1800])
b = np.array([2500,150,96,2300,29])
coef_func=np.array([[-0.8,-0.9,-2.7,-4,-6.5,-2.8],[2.5,4.8,32,32.4,14.6,13.8]])
"""
n=6
A = np.array([[216,164,456,516,176,670],[28,14,0,0,5,0],[2,8,0,0,0,0],[166,167,205,209,236,228],[2,2,8,12,8,13]])
d=np.array([1800,100,25,1300,22])
C = A
#d=np.array([100,22,1500,25,1800])
b = np.array([2500,150,36,2300,29])
coef_func=np.array([[-0.8,-0.9,-2.7,-4,-6.5,-2.8],[3,5,32,32,15,14]])
#
"""for i in range(36):
    print(f'Os valores das funções, com lb1 = {i+1} e lb2 = 1 é: ')
    soma_ponderada(n,A,b,C,d,coef_func,np.array([i+1,1]))"""

"for i in range(36):\n    print(f'Os valores das funções, com lb1 = {i+1} e lb2 = 1 é: ')\n    soma_ponderada(n,A,b,C,d,coef_func,np.array([i+1,1]))"

In [18]:
aaa = np.array([[1,2],[3,4],[5,6]])
aaa.shape[1]

2

In [69]:
from pulp import LpMaximize, LpProblem, LpStatus, LpVariable, lpDot
import numpy as np

def soma_ponderada(n, A=0, b=0, C=0, d=0, E=0, e=0, coef_func=0, vetor_lambda=0):
    # Definindo o problema de maximização
    model = LpProblem(name="multiobjective-weighted-sum", sense=LpMaximize)

    # Variáveis de decisão
    x = [LpVariable(name=f"x{i}", lowBound=0) for i in range(n)]

    # Adicionando restrições <=
    if isinstance(A, (list, np.ndarray)) and isinstance(b, (list, np.ndarray)):
        for i in range(len(A)):
            model += (lpDot(A[i], x) <= b[i], f"restrição_<=_{i}")

    # Adicionando restrições >=
    if isinstance(C, (list, np.ndarray)) and isinstance(d, (list, np.ndarray)):
        for i in range(len(C)):
            model += (lpDot(C[i], x) >= d[i], f"restrição_>=_{i}")

    # Adicionando restrições ==
    if isinstance(E, (list, np.ndarray)) and isinstance(e, (list, np.ndarray)):
        for i in range(len(E)):
            model += (lpDot(E[i], x) == e[i], f"restrição_==_{i}")

    # Função objetivo ponderada
    funcao_principal = lpDot(np.transpose(vetor_lambda) @ coef_func, x)
    model += funcao_principal

    # Resolvendo o problema
    status = model.solve()

    # Extraindo e exibindo a solução
    sol = np.array([x[i].varValue for i in range(n)])
    #print(f"Solução: {sol}")

    # Calculando os valores das funções objetivo
    value_func_aux = coef_func @ sol
    #for i in range(coef_func.shape[0]):
    #   print(f"O valor de f{i+1} é: {value_func_aux[i]}")

    #print(f"Status: {LpStatus[model.status]}")
    return sol,status

# Exemplo de uso
n = 6
A = np.array([
    [1, 1, 1, 0, 0, 0],  # x1A + x1B + x1C <= 100
    [0, 0, 0, 1, 1, 1]   # x2A + x2B + x2C <= 150
])
b = np.array([100, 150])

C = np.array([
    [1, 0, 0, 1, 0, 0],  # x1A + x2A >= 80
    [0, 1, 0, 0, 1, 0],  # x1B + x2B >= 70
    [0, 0, 1, 0, 0, 1]   # x1C + x2C >= 100
])
d = np.array([80, 70, 100])

coef_func = np.array([
    [2, 4, 5, 3, 1, 2],  # Coeficientes de z1
    [9, 4, 1, 2, 5, 8]   # Coeficientes de z2
])
vetor_lambda = np.array([0.5, 0.5])

# Executando o exemplo

soma_ponderada(n, A, b, C, d, E, e, coef_func, vetor_lambda)


(array([ 4.,  0., 96., 76., 70.,  4.]), 1)

ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [75]:
n = 6
A = np.array([
    [1, 1, 1, 0, 0, 0],  # x1A + x1B + x1C <= 100
    [0, 0, 0, 1, 1, 1]   # x2A + x2B + x2C <= 150
])
b = np.array([100, 150])

C = np.array([
    [1, 0, 0, 1, 0, 0],  # x1A + x2A >= 80
    [0, 1, 0, 0, 1, 0],  # x1B + x2B >= 70
    [0, 0, 1, 0, 0, 1]   # x1C + x2C >= 100
])
d = np.array([80, 70, 100])

coef_func = np.array([
    [2, 4, 5, 3, 1, 2],  # Coeficientes de z1
    [9, 4, 1, 2, 5, 8]   # Coeficientes de z2
])
vetor_lambda = np.array([0, 1])
soma_ponderada(n, A, b, C, d, 0, 0, coef_func, vetor_lambda)

(array([ 80.,  20.,   0.,   0.,  50., 100.]), 1)

In [76]:
def varrer_lambda():
    solutions = []
    lambdas = np.linspace(0, 90, 500)  # Variação de lambda de 0 a 1 com passo 0.1

    E =0 
    e=0
    for lambda1 in lambdas:
        lambda2 = np.random.uniform(0, lambda1) 
        vetor_lambda = np.array([lambda1, lambda2])
        sol, status = soma_ponderada(n, A, b, C, d, E, e, coef_func, vetor_lambda)

        if status == 1:
            solutions.append(tuple(np.round(sol, decimals=6)))  # Armazena a solução arredondada
            #print(vetor_lambda)
            #print(solutions)
    # Remover duplicatas
    solutions_distinct = list(set(solutions))
    
    return solutions_distinct

solucoes_distintas = varrer_lambda()

# Exibindo as soluções distintas
for sol in solucoes_distintas:
    print(f"Solução distinta: {sol}")

Solução distinta: (0.0, 70.0, 30.0, 80.0, 0.0, 70.0)
Solução distinta: (80.0, 0.0, 20.0, 0.0, 70.0, 80.0)
Solução distinta: (30.0, 70.0, 0.0, 50.0, 0.0, 100.0)
Solução distinta: (80.0, 20.0, 0.0, 0.0, 50.0, 100.0)


In [ ]:
space